We begin with setting up the Data Access Component (DAC). The DAC is responsible for retrieving the data we need as input for the MULTIPLY platform. During its setup, it will load several data stores. A data store manages access to data that is provided locally or remotely. The DAC can be asked about the stores it provides access to.

In [1]:
from multiply_data_access import DataAccessComponent
data_access_component = DataAccessComponent()
data_access_component.show_stores()

INFO:root:Read data store aws_s2
INFO:root:Read data store cams
INFO:root:Read data store emulators
INFO:root:Read data store Aster DEM
INFO:root:Read data store MODIS Data


Data store aws_s2
Data store cams
Data store emulators
Data store Aster DEM
Data store MODIS Data


We then can ask the Data Access Component what data of what type is available.

In [2]:
data_access_component.get_provided_data_types()

['AWS_S2_L1C',
 'CAMS',
 'ISO_MSI_A_EMU',
 'ISO_MSI_B_EMU',
 'WV_EMU',
 'AsterDEM',
 'MCD43A1.006']

Suppose we want to perform high resolution pre-processing. For this we need S2 L1C data in the format used by AWS (AWS_S2_L1C). We also need MODIS data (from 3 days before to 3 days after the S2 observations, data type MCD43A1.006), a global DEM file (AsterDEM) and several emulators for S2 data.
We also need to specify a spatial region of interest and start and end times. Let's suppose we are interested in the area around Barrax, Spain. We define the area by passing its coordinates in Well-Known-Text-Format (you can use https://arthur-e.github.io/Wicket/sandbox-gmaps3.html to get WKT representations of other regions of interest).
We also define a start and ending time.
When we have done that, we can ask the data access component for available S2 data.

In [3]:
BARRAX_ROI = "POLYGON((-2.20397502663252 39.09868106889479,-1.9142106223355313 39.09868106889479," \
             "-1.9142106223355313 38.94504502508093,-2.20397502663252 38.94504502508093," \
             "-2.20397502663252 39.09868106889479))"
start_time = '2017-01-01'
end_time = '2017-01-20'
s2_data_infos = data_access_component.query(BARRAX_ROI, start_time, end_time, 'AWS_S2_L1C')
s2_data_infos


[Data Set:
   Id: 30/S/WJ/2017/1/16/0, 
   Type: AWS_S2_L1C, 
   Start Time: 2017-01-16, 
   End Time: 2017-01-16, 
   Coverage: POLYGON((-3.00023345437724 39.7502679265611,-3.00023019602957 38.7608644567253,-1.73659678081167 38.7540360477761,-1.71871965133358 39.7431961916792,-3.00023345437724 39.7502679265611)),
 Data Set:
   Id: 30/S/WJ/2017/1/19/0, 
   Type: AWS_S2_L1C, 
   Start Time: 2017-01-19, 
   End Time: 2017-01-19, 
   Coverage: POLYGON((-3.00023345437724 39.7502679265611,-3.00023019602957 38.7608644567253,-1.73659678081167 38.7540360477761,-1.71871965133358 39.7431961916792,-3.00023345437724 39.7502679265611))]

This has retrieved meta information about available S2 data. In particular, we can see the data coverage and the sensing date. When we are happy with the selection, let's get the data:

In [4]:
s2_urls = data_access_component.get_data_urls_from_data_set_meta_infos(s2_data_infos)

INFO:root:Downloading S2 Data from 1-16-2017
INFO:root:Downloaded S2 Data from 1-16-2017
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.amazonaws.com
INFO:botocore.vendored.requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): sentinel-s2-l1c.s3.a

Now, let's get all the other data we need. First, all the emulators. We need to download them only once and can re-use them later.

In [6]:
emu_urls = data_access_component.get_data_urls(BARRAX_ROI, start_time, end_time, 'ISO_MSI_A_EMU,ISO_MSI_B_EMU,WV_EMU')

INFO:root:Downloading isotropic_MSI_emulators_correction_xap_S2A.pkl


99 %

INFO:root:Downloaded isotropic_MSI_emulators_correction_xap_S2A.pkl
INFO:root:Downloading isotropic_MSI_emulators_correction_xbp_S2A.pkl


99 %

INFO:root:Downloaded isotropic_MSI_emulators_correction_xbp_S2A.pkl
INFO:root:Downloading isotropic_MSI_emulators_correction_xcp_S2A.pkl


...

INFO:root:Downloaded isotropic_MSI_emulators_optimization_xbp_S2B.pkl
INFO:root:Downloading isotropic_MSI_emulators_optimization_xcp_S2B.pkl


100 %

INFO:root:Downloaded isotropic_MSI_emulators_optimization_xcp_S2B.pkl
INFO:root:Downloading wv_MSI_retrieval_S2A.pkl


99 %

INFO:root:Downloaded wv_MSI_retrieval_S2A.pkl


Let's get the ASTER DEM URL next ...

In [8]:
aster_dem_url = data_access_component.get_data_urls(BARRAX_ROI, start_time, end_time, 'AsterDEM')

INFO:root:Downloading ASTGTM2_N38W002_dem.tif


99 %

INFO:root:Downloaded ASTGTM2_N38W002_dem.tif
INFO:root:Downloading ASTGTM2_N38W003_dem.tif


99 %

INFO:root:Downloaded ASTGTM2_N38W003_dem.tif
INFO:root:Downloading ASTGTM2_N39W002_dem.tif


99 %

INFO:root:Downloaded ASTGTM2_N39W002_dem.tif
INFO:root:Downloading ASTGTM2_N39W003_dem.tif


99 %

INFO:root:Downloaded ASTGTM2_N39W003_dem.tif


For CAMS, we only need the data for the days for which we have S2 observations.

In [9]:
cams_urls = data_access_component.get_data_urls(BARRAX_ROI, '2017-1-16', '2017-1-16', 'CAMS')
cams_urls.extend(data_access_component.get_data_urls(BARRAX_ROI, '2017-1-19', '2017-1-19', 'CAMS'))

INFO:root:Downloading 2017-01-16.nc


100 %

INFO:root:Downloaded 2017-01-16.nc
INFO:root:Downloading 2017-01-19.nc


100 %

INFO:root:Downloaded 2017-01-19.nc
